# Counter DApp

In [1]:
%%writefile src/Counter.sol
pragma solidity ^0.4.0;
contract Counter {
    uint256 counter = 0;
    function add() public {
        counter++;
    }
    function subtract() public {
        counter--;
    }
    function getCounter() public view returns (uint256) {
        return counter;
    }
}

Writing src/Counter.sol


In [3]:
!solc --optimize --combined-json abi,bin src/Counter.sol > src/Counter.json

In [4]:
!type src\Counter.json

{"contracts":{"src/Counter.sol:Counter":{"abi":"[{\"constant\":false,\"inputs\":[],\"name\":\"add\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"subtract\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getCounter\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"60806040526000805534801561001457600080fd5b5060ea806100236000396000f30060806040526004361060525763ffffffff7c01000000000000000000000000000000000000000000000000000000006000350416634f2be91f811460575780636deebae314606b5780638ada066e14607d575b600080fd5b348015606257600080fd5b50606960a1565b005b348015607657600080fd5b50606960ac565b348015608857600080fd5b50608f60b8565b60408051918252519081900360200190f35b600080546001019055565b60008054600019019055565b600054905600a165627a7a72305820bf086

In [5]:
%%writefile src/CounterDeployAbiBinFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Counter.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Timer.sol:Timer']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
_bin=_abiBinJson.contracts[contractName].bin;

console.log("- ABI: " + _abi);
console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/CounterDeployAbiBinFromFile.js


In [29]:
!node src/CounterDeployAbiBinFromFile.js

- contract name:  [ 'src/Counter.sol:Counter' ]
- ABI: [{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}]
- Bytecode: 60806040526000805534801561001457600080fd5b5060ea806100236000396000f30060806040526004361060525763ffffffff7c01000000000000000000000000000000000000000000000000000000006000350416634f2be91f811460575780636deebae314606b5780638ada066e14607d575b600080fd5b348015606257600080fd5b50606960a1565b005b348015607657600080fd5b50606960ac565b348015608857600080fd5b50608f60b8565b60408051918252519081900360200190f35b600080546001019055565b60008054600019019055565b600054905600a165627a7a72305820bf086d20c7eb5431a642475bfd5137118a0044a6cbf4e1ce2cc17b5fe112db

In [66]:
%%writefile src/counterUse.js
var Web3=require('web3');
var _abiBinJson = require('./Counter.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);
var counter = new web3.eth.Contract(_abiArray,"0x810C6102d3a3e05B5a596022Bb4873514ea5f5Cc");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Call from: " + accounts[0]);
    counter.methods.getCounter().call().then(function(str) {console.log(str);});
    await counter.methods.add().send({from:accounts[0],gas:100000});
    await counter.methods.add().send({from:accounts[0],gas:100000});
    counter.methods.getCounter().call().then(function(str) {console.log(str);});
}
doIt()

Overwriting src/counterUse.js


In [67]:
!node src/counterUse.js

Call from: 0x7455952108B0E6e559411B4B39134ff90b96Dbc7
6
8


http 서버 열기
python -m http.server 8045

In [37]:
%%writefile scripts/simpleCounter.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script>
<!-- script src="https://cdn.jsdelivr.net/npm/web3@0.20.5/dist/web3.min.js"></script -->
<script type="text/javascript">
    //var Web3 = require('web3');  //can be removed
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
    

    function displayCounter() {
        var web3version = web3.version;
        document.getElementById('web3ver').innerText = 'web3 version: '+web3version;

        var counterContract = new web3.eth.Contract([{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}],
                                            "0x810C6102d3a3e05B5a596022Bb4873514ea5f5Cc"); 
        counterContract.methods.getCounter().call().then(function(b) {
            document.getElementById('bc').innerText = "b: "+b;
        });
        
        counterContract.methods.getCounter().call().then(function(a) {
            document.getElementById('ac').innerText = "a: "+a;
        });
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayCounter();">Display Counter</button>
    <div></div>
    <div id="web3ver"></div>
    <div id="bc"></div>
    <div id="ac"></div>
</body>
</html>


Overwriting scripts/simpleCounter.html


In [38]:
%%writefile scripts/simpleCounter.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script>
<!-- script src="https://cdn.jsdelivr.net/npm/web3@0.20.5/dist/web3.min.js"></script -->
<script type="text/javascript">
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
    console.log("Hi")
    
    async function displayCounter() {
        const accounts = await web3.eth.getAccounts();
        var counter = new web3.eth.Contract([{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}],
                                            "0x810C6102d3a3e05B5a596022Bb4873514ea5f5Cc"); 
        counter.methods.getCounter().call().then(function(str) {
            document.getElementById('C1').innerText = "Before : " + str;
        });
        await counter.methods.add().send({from:accounts[0],gas:100000});
        await counter.methods.add().send({from:accounts[0],gas:100000});
        counter.methods.getCounter().call().then(function(str) {
            document.getElementById('C2').innerText = "After : " + str;
        });
        
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayCounter();">Display Counter</button>
    <div></div>
    <div id="C1"></div>
    <div id="C2"></div>
</body>
</html>

Overwriting scripts/simpleCounter.html


# Counter + Timer

In [39]:
%%writefile src/TimerCounter.sol
pragma solidity ^0.4.0;
contract Timer {
    uint256 startTime;
    uint256 counter = 0;
    function start() public {
        //startTime=now; now is deprecated as of v0.7
        startTime=block.timestamp;
    }
    function timePassed() public view returns(uint256) {
        //return now-startTime;
        return block.timestamp-startTime;
    }
    function getNow() view public returns(uint) {
        //return now;
        return block.timestamp;
    }
    function add() public {
        counter++;
    }
    function subtract() public {
        counter--;
    }
    function getCounter() public view returns (uint256) {
        return counter;
    }
}

Writing src/TimerCounter.sol


In [41]:
!solc --optimize --combined-json abi,bin src/TimerCounter.sol > src/TimerCounter.json

In [43]:
!type src\TimerCounter.json

{"contracts":{"src/TimerCounter.sol:Timer":{"abi":"[{\"constant\":false,\"inputs\":[],\"name\":\"add\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"subtract\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getCounter\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\"

In [44]:
%%writefile src/TimerCounterDeployAbiBinFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./TimerCounter.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
_bin=_abiBinJson.contracts[contractName].bin;

console.log("- ABI: " + _abi);
console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/TimerCounterDeployAbiBinFromFile.js


In [45]:
!node src/TimerCounterDeployAbiBinFromFile.js

- contract name:  [ 'src/TimerCounter.sol:Timer' ]
- ABI: [{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"timePassed","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getNow","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"start","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]
- Bytecode: 6080604052600060015534801561001557600080fd5b50610168806100256000396000f3006080604052600436106100775763fffffff

In [64]:
%%writefile src/timerCounterUse.js
var Web3=require('web3');
var _abiBinJson = require('./TimerCounter.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);
var TC = new web3.eth.Contract(_abiArray,"0x2d0a3808593D3EF9cFE89f794B568360CBb77d9E");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Call from: " + accounts[0]);
    var start = await TC.methods.getNow().call();
    console.log("start time: "+ start);
    TC.methods.getCounter().call().then(function(str) {console.log("before: "+str);});
    await TC.methods.add().send({from:accounts[0],gas:100000});
    await TC.methods.add().send({from:accounts[0],gas:100000});
    await TC.methods.add().send({from:accounts[0],gas:100000});
    var stop = await TC.methods.getNow().call();
    console.log("stop time: "+ stop);
    TC.methods.getCounter().call().then(function(str) {console.log("after: "+str);});
    var time = await stop - start;
    console.log("time: "+ time);
    
}
doIt()

Overwriting src/timerCounterUse.js


In [65]:
!node src/timerCounterUse.js

Call from: 0x7455952108B0E6e559411B4B39134ff90b96Dbc7
start time: 1650719752
before: 9
stop time: 1650719818
time: 66
after: 12
